In [1]:
%load_ext autoreload

In [2]:
%autoreload

import os, pandas as pd
import stanza
import nltk

RESOURCE_PATH = "/mnt/projects/OpenNRE/benchmark/raw_ddi/DDICorpus"
OUTPUT_PATH = "/mnt/projects/OpenNRE/benchmark/ddi"
outdir = 'original/'
indir = 'original/'
outdir1 = 'entity_blinding/'
outdir2 = 'punct_stop_digit/'
outdir3 = 'punct_digit/'
outdir4 = 'ner_blinding/'
def res(path): return os.path.join(RESOURCE_PATH, path)
def out(path): return os.path.join(OUTPUT_PATH, path)
from opennre.dataset.converters.converter_ddi import get_dataset_dataframe, write_dataframe, \
read_dataframe, check_equality_of_written_and_read_df, write_into_txt, combine, write_relations_json
from opennre.dataset.preprocess import preprocess, replace_ner, get_entity_positions_and_replacement_sentence

def makedir(outdir, res):
    if not os.path.exists(res(outdir)):
        os.makedirs(res(outdir))

[nltk_data] Downloading package stopwords to /home/igor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
stanza.download('en', package='craft')
nlp = stanza.Pipeline('en', package='craft')

2021-06-19 01:05:07 INFO: Downloading these customized packages for language: en (English)...
| Processor | Package |
-----------------------
| tokenize  | craft   |
| pos       | craft   |
| lemma     | craft   |
| depparse  | craft   |
| pretrain  | craft   |

2021-06-19 01:05:07,358 - stanza - INFO - Downloading these customized packages for language: en (English)...
| Processor | Package |
-----------------------
| tokenize  | craft   |
| pos       | craft   |
| lemma     | craft   |
| depparse  | craft   |
| pretrain  | craft   |

2021-06-19 01:05:07 INFO: File exists: /home/igor/stanza_resources/en/tokenize/craft.pt.
2021-06-19 01:05:07,364 - stanza - INFO - File exists: /home/igor/stanza_resources/en/tokenize/craft.pt.
2021-06-19 01:05:07 INFO: File exists: /home/igor/stanza_resources/en/pos/craft.pt.
2021-06-19 01:05:07,420 - stanza - INFO - File exists: /home/igor/stanza_resources/en/pos/craft.pt.
2021-06-19 01:05:07 INFO: File exists: /home/igor/stanza_resources/en/lemma/craf

In [4]:
df_train_drugbank = get_dataset_dataframe(nlp, res('Train/DrugBank/'), relation_extraction=True)

100%|██████████| 572/572 [43:50<00:00,  4.60s/it]


In [5]:
df_train_medline = get_dataset_dataframe(nlp, res('Train/MedLine/'), relation_extraction=True)

100%|██████████| 142/142 [01:38<00:00,  1.44it/s]


In [6]:
df_test_drugbank = get_dataset_dataframe(nlp, res('Test/Test for DDI Extraction task/DrugBank/'), relation_extraction=True)

100%|██████████| 158/158 [06:51<00:00,  2.60s/it]


In [7]:
df_test_medline = get_dataset_dataframe(nlp, res('Test/Test for DDI Extraction task/MedLine/'), relation_extraction=True)

100%|██████████| 33/33 [00:22<00:00,  1.43it/s]


In [8]:
len(df_train_drugbank)

26004

In [9]:
len(df_train_medline)

1787

In [10]:
len(df_test_drugbank)

5265

In [11]:
len(df_test_medline)

451

In [12]:
def get_empty_entity_rows(df):
    empty_entity_rows = []
    def find_empty_entity_number(row):
        metadata = row.metadata
        e1 = metadata['e1']['word_index']
        e2 = metadata['e2']['word_index']
        if not e1 or not e2:
            empty_entity_rows.append(row.row_num)
    temp_df = df.copy()
    temp_df.insert(0, 'row_num', range(0, len(temp_df)))
    temp_df.apply(find_empty_entity_number, axis=1)
    return empty_entity_rows

def get_empty_rows_array(empty_entity_rows, df):
    empty_rows_array = []
    for index in empty_entity_rows:
        e1 = df.iloc[index].e1
        e2 = df.iloc[index].e2
        original_sentence = df.iloc[index].original_sentence
        tokenized_sentence = df.iloc[index].tokenized_sentence
        metadata = df.iloc[index].metadata
        empty_rows_array.append([index, original_sentence, e1, e2, metadata, tokenized_sentence])
    new_df = pd.DataFrame(data=empty_rows_array,    # values
             columns=['index_original', 'original_sentence' , 'e1', 'e2', 'metadata', 'tokenized_sentence'])
    return empty_rows_array, new_df

In [13]:
def get_empty_vals(df):
    empty_entity_rows = get_empty_entity_rows(df)
    empty_rows_array, new_df = get_empty_rows_array(empty_entity_rows, df)
    return empty_rows_array, new_df

In [14]:
get_empty_vals(df_train_drugbank)

([],
 Empty DataFrame
 Columns: [index_original, original_sentence, e1, e2, metadata, tokenized_sentence]
 Index: [])

In [15]:
get_empty_vals(df_train_medline)

([],
 Empty DataFrame
 Columns: [index_original, original_sentence, e1, e2, metadata, tokenized_sentence]
 Index: [])

In [16]:
get_empty_vals(df_test_drugbank)

([],
 Empty DataFrame
 Columns: [index_original, original_sentence, e1, e2, metadata, tokenized_sentence]
 Index: [])

In [17]:
get_empty_vals(df_test_medline)

([],
 Empty DataFrame
 Columns: [index_original, original_sentence, e1, e2, metadata, tokenized_sentence]
 Index: [])

In [18]:
if not os.path.exists(out(outdir)):
    os.makedirs(out(outdir))

In [19]:
write_dataframe(df_train_drugbank, out(outdir + 'train_drugbank_original.csv'))

In [20]:
df_train_drugbank_copy = read_dataframe(out(outdir + 'train_drugbank_original.csv'))

In [21]:
# The first checks with the pd.equals method, and the other does a manual checking per column
check_equality_of_written_and_read_df(df_train_drugbank, df_train_drugbank_copy)

(True, True)

In [22]:
write_dataframe(df_train_medline, out(outdir + 'train_medline_original.csv'))

In [23]:
df_train_medline_copy = read_dataframe(out(outdir + 'train_medline_original.csv'))

In [24]:
check_equality_of_written_and_read_df(df_train_medline, df_train_medline_copy)

(True, True)

In [25]:
write_dataframe(df_test_drugbank, out(outdir + 'test_drugbank_original.csv'))

In [26]:
df_test_drugbank_copy = read_dataframe(out(outdir + 'test_drugbank_original.csv'))

In [27]:
check_equality_of_written_and_read_df(df_test_drugbank, df_test_drugbank_copy)

(True, True)

In [28]:
write_dataframe(df_test_medline, out(outdir + 'test_medline_original.csv'))

In [29]:
df_test_medline_copy = read_dataframe(out(outdir + 'test_medline_original.csv'))

In [30]:
check_equality_of_written_and_read_df(df_test_medline, df_test_medline_copy)

(True, True)

In [31]:
write_into_txt(df_train_drugbank, out(outdir + 'train_drugbank_original.txt'))

100%|██████████| 26004/26004 [00:03<00:00, 8497.92it/s]


In [32]:
write_into_txt(df_train_medline, out(outdir + 'train_medline_original.txt'))

100%|██████████| 1787/1787 [00:00<00:00, 8262.36it/s]


In [33]:
write_into_txt(df_test_drugbank, out(outdir + 'test_drugbank_original.txt'))

100%|██████████| 5265/5265 [00:00<00:00, 7885.13it/s]


In [34]:
write_into_txt(df_test_medline, out(outdir + 'test_medline_original.txt'))

100%|██████████| 451/451 [00:00<00:00, 8103.49it/s]Unique relations: 	 ['none' 'mechanism' 'advise' 'effect' 'int']



In [35]:
combine(out, outdir,  'train_drugbank_original', 'train_medline_original', 'ddi_train_original.txt')

In [36]:
combine(out, outdir, 'test_drugbank_original', 'test_medline_original', 'ddi_val_original.txt')

In [37]:
original_dataframe_names = ['train_drugbank', 'train_medline', 'test_drugbank', 'test_medline']
makedir(outdir1, out)
makedir(outdir2, out)
makedir(outdir3, out)
makedir(outdir4, out)

In [38]:
stanza.download('en', package='craft', processors={'ner': 'bionlp13cg'})
nlp = stanza.Pipeline('en', package="craft", processors={"ner": "bionlp13cg"}, tokenize_no_ssplit=True)

2021-06-19 01:58:10 INFO: Downloading these customized packages for language: en (English)...
| Processor       | Package    |
--------------------------------
| tokenize        | craft      |
| pos             | craft      |
| lemma           | craft      |
| depparse        | craft      |
| ner             | bionlp13cg |
| pretrain        | craft      |
| backward_charlm | pubmed     |
| forward_charlm  | pubmed     |

2021-06-19 01:58:10,202 - stanza - INFO - Downloading these customized packages for language: en (English)...
| Processor       | Package    |
--------------------------------
| tokenize        | craft      |
| pos             | craft      |
| lemma           | craft      |
| depparse        | craft      |
| ner             | bionlp13cg |
| pretrain        | craft      |
| backward_charlm | pubmed     |
| forward_charlm  | pubmed     |

2021-06-19 01:58:10 INFO: File exists: /home/igor/stanza_resources/en/tokenize/craft.pt.
2021-06-19 01:58:10,206 - stanza - INFO - Fil

In [39]:
for original_df_name in original_dataframe_names:
    type1 = preprocess(read_dataframe, out(indir + original_df_name + '_original.csv'), nlp)
    type2 = preprocess(read_dataframe, out(indir + original_df_name + '_original.csv'), nlp, 2)
    type3 = preprocess(read_dataframe, out(indir + original_df_name + '_original.csv'), nlp, 3)
    type4 = preprocess(read_dataframe, out(indir + original_df_name + '_original.csv'), nlp, 4)
    write_dataframe(type1, out(outdir1 + original_df_name + '_entity_blinding.csv'))
    write_dataframe(type2, out(outdir2 + original_df_name + '_punct_stop_digit.csv'))
    write_dataframe(type3, out(outdir3 + original_df_name + '_punct_digit.csv'))
    write_dataframe(type4, out(outdir4 + original_df_name + '_ner_blinding.csv'))

In [40]:
for original_df_name in original_dataframe_names:
    type1 = read_dataframe(out(outdir1 + original_df_name + '_entity_blinding.csv'))
    type2 = read_dataframe(out(outdir2 + original_df_name + '_punct_stop_digit.csv'))
    type3 = read_dataframe(out(outdir3 + original_df_name + '_punct_digit.csv'))
    type4 = read_dataframe(out(outdir4 + original_df_name + '_ner_blinding.csv'))
    write_into_txt(type1, out(outdir1 + original_df_name + '_entity_blinding.txt'))
    write_into_txt(type2, out(outdir2 + original_df_name + '_punct_stop_digit.txt'))
    write_into_txt(type3, out(outdir3 + original_df_name + '_punct_digit.txt'))
    write_into_txt(type4, out(outdir4 + original_df_name + '_ner_blinding.txt'))

  0%|          | 0/451 [00:00<?, ?it/s]Unique relations: 	 ['none' 'mechanism' 'advise' 'effect' 'int']
Unique relations: 	 ['none' 'mechanism' 'advise' 'effect' 'int']
Unique relations: 	 ['none' 'mechanism' 'advise' 'effect' 'int']
Unique relations: 	 ['none' 'mechanism' 'advise' 'effect' 'int']
100%|██████████| 451/451 [00:00<00:00, 8152.70it/s]


In [47]:
for i in ['train', 'test']:
    if i == 'train':
        combine(out, outdir1, i + '_drugbank_entity_blinding', i + '_medline_entity_blinding', 'ddi_train_entity_blinding.txt')
        combine(out, outdir2, i + '_drugbank_punct_stop_digit', i + '_medline_punct_stop_digit', 'ddi_train_punct_stop_digit.txt')
        combine(out, outdir3, i + '_drugbank_punct_digit', i + '_medline_punct_digit', 'ddi_train_punct_digit.txt')
        combine(out, outdir4, i + '_drugbank_ner_blinding', i + '_medline_ner_blinding', 'ddi_train_ner_blinding.txt')
    else:
        combine(out, outdir1, i + '_drugbank_entity_blinding', i + '_medline_entity_blinding', 'ddi_val_entity_blinding.txt')
        combine(out, outdir2, i + '_drugbank_punct_stop_digit', i + '_medline_punct_stop_digit', 'ddi_val_punct_stop_digit.txt')
        combine(out, outdir3, i + '_drugbank_punct_digit', i + '_medline_punct_digit', 'ddi_val_punct_digit.txt')
        combine(out, outdir4, i + '_drugbank_ner_blinding', i + '_medline_ner_blinding', 'ddi_val_ner_blinding.txt')

In [48]:
write_relations_json(out(''))

In [49]:
def output_file_length(res, filename):
    return len(open(res(filename)).readlines())

In [50]:
print(output_file_length(out, indir + 'ddi_train_original.txt'))
print(output_file_length(out, outdir1 + 'train_entity_blinding.txt'))
print(output_file_length(out, outdir2 + 'train_punct_stop_digit.txt'))
print(output_file_length(out, outdir3 + 'train_punct_digit.txt'))
print(output_file_length(out, outdir4 + 'train_ner_blinding.txt'))

27791
27791
27791
27791
27791


In [55]:
print(output_file_length(out, indir + 'ddi_val_original.txt'))
print(output_file_length(out, outdir1 + 'ddi_val_entity_blinding.txt'))
print(output_file_length(out, outdir2 + 'ddi_val_punct_stop_digit.txt'))
print(output_file_length(out, outdir3 + 'ddi_val_punct_digit.txt'))
print(output_file_length(out, outdir4 + 'ddi_val_ner_blinding.txt'))

5716
5716
5716
5716
5716
